#### Code review.
#### Привет (:
##### 1. Логи слишком большие, тяжеловато ретривить код из всей информации. Я бы не вставляла для анализа.
При сode review преимущественно анализируется техническое исполнение, масштабируемость и поддерживаемость кода.
##### 2. Вместо varchar лучше использовать string. У string нет ограничения на размерность и лучше перфоманс.
##### 3. COUNT_ можно назвать, например, cnt_visits_10. Ну или как-то иначе, чтобы можно было понять, что считалось. Тоже самое и для других (first_cat = max_1st_cat).
##### 4. Лучше всегда указывать принадлежность колонки (In [59]). Сейчас один источник, но, например, если код будет расширять другой разработчик и добавит таблицу с такими же колонками, понадобится больше времени на правку кода. (:
##### 5. Для колонок один регистр лучше использовать. Визуально приятнее.
##### 6. Колонкам основных объектов лучше ассанить комментарии (uri BIGINT comment 'User identificator',...).
Если будет непонятное название для кого-то, то по комментарию легче будет разобраться.
##### 7. In [47] и In [59] можно смело объединить.
#### Хорошего вечера. (:


In [1]:
import pandas as pd
import numpy as np

from urllib.parse import urlparse, unquote
import re
from urllib.parse import urlparse, unquote

def url2domain(url):
   try:
       a = urlparse(unquote(url.strip()))
       if (a.scheme in ['http','https']):
           b = re.search("(?:www\.)?(.*)",a.netloc).group(1)
           if b is not None:
               return str(b).strip()
           else:
               return ''
       else:
           return ''
   except:
       return ''

# Part I

### Put raw data:

In [8]:
from sqlalchemy import create_engine

#Input Information
host = "bd-node1.newprolab.com"
port = 10000

pwd = pwd
login = username

In [17]:
raw_data_path = "/user/viktorya.mukukenova/result_lab03_1"
raw_hive_tbl_data_path = "/tmp/viktorya.mukukenova/result_lab03_3"

create_cmd_line = f"hdfs dfs -mkdir {raw_hive_tbl_data_path}"
cp_cmd_line = f"hdfs dfs -cp {raw_data_path} {raw_hive_tbl_data_path}"

!$create_cmd_line
!$cp_cmd_line

In [19]:
create_hive_tbl = f""" use db_vm;
CREATE EXTERNAL TABLE db_vm.raw_data
(
 URI BIGINT,  URL VARCHAR(1000)
)
row format delimited
fields terminated by '\t'
STORED AS TEXTFILE
LOCATION '{raw_hive_tbl_data_path}/result_lab03_1/';"""

create_script_path = "lab03_create_raw_tbl.sql"
with open(create_script_path, 'w') as file_:
    file_.writelines(create_hive_tbl) 

In [20]:
cmd_line = f"beeline jdbc:hive2://{host}:{port} -n {login} -p {pwd} -f {create_script_path}"
!$cmd_line

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Connecting to jdbc:hive2://bd-master.newprolab.com:2181,bd-node1.newprolab.com:2181,bd-node2.newprolab.com:2181/default;password=dczcdGv5;serviceDiscoveryMode=zooKeeper;user=viktorya.mukukenova;zooKeeperNamespace=hiveserver2
20/10/24 02:30:44 [main]: INFO jdbc.HiveConnection: Connected to bd-node2.newprolab.com:10000
Connected to: Apache Hive (version 3.1.0.3.1.4.0-315)
Driver: Hive JDBC (version 3.1.0.3.1.4.0-315)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://bd-master.newprolab.com:2181,>  use

### Create group table with important data:

In [45]:
grp_hive_tbl_data_path = "/tmp/viktorya.mukukenova/result_lab03_3/result_lab03_1_grp"

create_cmd_line = f"hdfs dfs -mkdir {grp_hive_tbl_data_path}"

!$create_cmd_line

In [46]:
create_hive_tbl = f""" use db_vm;
CREATE EXTERNAL TABLE db_vm.grp_data
(
 URI BIGINT,  URL VARCHAR(1000), COUNT_ BIGINT
)
row format delimited
fields terminated by '\t'
STORED AS TEXTFILE
LOCATION '{grp_hive_tbl_data_path}';"""

create_script_path = "lab03_create_grp_tbl.sql"
with open(create_script_path, 'w') as file_:
    file_.writelines(create_hive_tbl) 
    
cmd_line = f"beeline jdbc:hive2://{host}:{port} -n {login} -p {pwd} -f {create_script_path}"
!$cmd_line

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Connecting to jdbc:hive2://bd-master.newprolab.com:2181,bd-node1.newprolab.com:2181,bd-node2.newprolab.com:2181/default;password=dczcdGv5;serviceDiscoveryMode=zooKeeper;user=viktorya.mukukenova;zooKeeperNamespace=hiveserver2
20/10/24 03:38:41 [main]: INFO jdbc.HiveConnection: Connected to bd-node2.newprolab.com:10000
Connected to: Apache Hive (version 3.1.0.3.1.4.0-315)
Driver: Hive JDBC (version 3.1.0.3.1.4.0-315)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://bd-master.newprolab.com:2181,>  use

#### Insert into grp table:

In [47]:
insert_hive_tbl = f""" use db_vm;
INSERT INTO db_vm.grp_data
SELECT URI, URL, COUNT(*) as COUNT_
FROM db_vm.raw_data
WHERE URL in ('cars.ru', 'avto-russia.ru', 'bmwclub.ru', 'vk.com', 'mail.qip.ru', 'lk.ssl.mts.ru',
    'games.mail.ru', 'igrystrelyalki.ru', 'consol-games.net', '5ballov.qip.ru', 'mgimo.ru', 'referat.arxiv.uz')
    GROUP BY URI, URL
    HAVING COUNT_ >= 10 ;
"""

insert_script_path = "lab03_insert_grp_tbl.sql"
with open(insert_script_path, 'w') as file_:
    file_.writelines(insert_hive_tbl) 
    
cmd_line = f"beeline jdbc:hive2://{host}:{port} -n {login} -p {pwd} -f {insert_script_path}"
!$cmd_line

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Connecting to jdbc:hive2://bd-master.newprolab.com:2181,bd-node1.newprolab.com:2181,bd-node2.newprolab.com:2181/default;password=dczcdGv5;serviceDiscoveryMode=zooKeeper;user=viktorya.mukukenova;zooKeeperNamespace=hiveserver2
20/10/24 03:39:06 [main]: INFO jdbc.HiveConnection: Connected to bd-node2.newprolab.com:10000
Connected to: Apache Hive (version 3.1.0.3.1.4.0-315)
Driver: Hive JDBC (version 3.1.0.3.1.4.0-315)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://bd-master.newprolab.com:2181,>  use

----------------------------------------------------------------------------------------------
        VERTICES      MODE        STATUS  TOTAL  COMPLETED  RUNNING  PENDING  FAILED  KILLED  
----------------------------------------------------------------------------------------------
Map 1 ....       container       RUNNING     10          4        6        0       0       0  
Reducer 2        container        INITED      2          0        0        2       0       0  
Reducer 3        container        INITED      1          0        0        1       0       0  
----------------------------------------------------------------------------------------------
VERTICES: 00/03  [========>>------------------] 30%   ELAPSED TIME: 8.02 s     
----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------
        VERTICES      MODE        STATUS  TOTAL  COMPLETED  RUNNI

0: jdbc:hive2://bd-master.newprolab.com:2181,> Closing: 0: jdbc:hive2://bd-master.newprolab.com:2181,bd-node1.newprolab.com:2181,bd-node2.newprolab.com:2181/default;password=dczcdGv5;serviceDiscoveryMode=zooKeeper;user=viktorya.mukukenova;zooKeeperNamespace=hiveserver2


### Create unique URI  grp table:

In [57]:
grp_hive_tbl_data_path = "/tmp/viktorya.mukukenova/result_lab03_3/result_lab03_1_grp_unique"

create_cmd_line = f"hdfs dfs -mkdir {grp_hive_tbl_data_path}"

!$create_cmd_line

In [58]:
create_hive_tbl = f""" use db_vm;
CREATE EXTERNAL TABLE db_vm.grp_data_unique
(
 URI BIGINT, first_cat BIGINT, second_cat BIGINT, thirst_cat BIGINT, fourth_cat BIGINT
)
row format delimited
fields terminated by '\t'
STORED AS TEXTFILE
LOCATION '{grp_hive_tbl_data_path}';"""

create_script_path = "lab03_create_grp_unique_tbl.sql"
with open(create_script_path, 'w') as file_:
    file_.writelines(create_hive_tbl) 
    
cmd_line = f"beeline jdbc:hive2://{host}:{port} -n {login} -p {pwd} -f {create_script_path}"
!$cmd_line

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Connecting to jdbc:hive2://bd-master.newprolab.com:2181,bd-node1.newprolab.com:2181,bd-node2.newprolab.com:2181/default;password=dczcdGv5;serviceDiscoveryMode=zooKeeper;user=viktorya.mukukenova;zooKeeperNamespace=hiveserver2
20/10/24 03:56:26 [main]: INFO jdbc.HiveConnection: Connected to bd-node1.newprolab.com:10000
Connected to: Apache Hive (version 3.1.0.3.1.4.0-315)
Driver: Hive JDBC (version 3.1.0.3.1.4.0-315)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://bd-master.newprolab.com:2181,>  use

In [59]:
insert_hive_tbl = f"""use db_vm; 
INSERT INTO db_vm.grp_data_unique
SELECT A.URI as URI, 
MAX(first_cat_) as first_cat, 
MAX(second_cat_) as second_cat, 
MAX(thirst_cat_) as thirst_cat, 
MAX(fourth_cat_) as fourth_cat
FROM
(SELECT URI, 
CASE WHEN URL IN ('cars.ru', 'avto-russia.ru', 'bmwclub.ru') THEN 1 ELSE 0 END first_cat_,
CASE WHEN URL IN ('vk.com', 'mail.qip.ru', 'lk.ssl.mts.ru') THEN 1 ELSE 0 END second_cat_,
CASE WHEN URL IN ('games.mail.ru', 'igrystrelyalki.ru', 'consol-games.net') THEN 1 ELSE 0 END thirst_cat_,
CASE WHEN URL IN ('5ballov.qip.ru', 'mgimo.ru', 'referat.arxiv.uz') THEN 1 ELSE 0 END fourth_cat_
FROM db_vm.grp_data) A
GROUP BY URI;
"""

insert_script_path = "lab03_insert_grp_unique_tbl.sql"
with open(insert_script_path, 'w') as file_:
    file_.writelines(insert_hive_tbl) 
    
cmd_line = f"beeline jdbc:hive2://{host}:{port} -n {login} -p {pwd} -f {insert_script_path}"
!$cmd_line

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Connecting to jdbc:hive2://bd-master.newprolab.com:2181,bd-node1.newprolab.com:2181,bd-node2.newprolab.com:2181/default;password=dczcdGv5;serviceDiscoveryMode=zooKeeper;user=viktorya.mukukenova;zooKeeperNamespace=hiveserver2
20/10/24 03:56:31 [main]: INFO jdbc.HiveConnection: Connected to bd-node2.newprolab.com:10000
Connected to: Apache Hive (version 3.1.0.3.1.4.0-315)
Driver: Hive JDBC (version 3.1.0.3.1.4.0-315)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://bd-master.newprolab.com:2181,> use 

----------------------------------------------------------------------------------------------
        VERTICES      MODE        STATUS  TOTAL  COMPLETED  RUNNING  PENDING  FAILED  KILLED  
----------------------------------------------------------------------------------------------
Map 1 .....      container       RUNNING      2          1        1        0       0       0  
Reducer 2        container       RUNNING      2          0        2        0       0       0  
Reducer 3        container        INITED      1          0        0        1       0       0  
----------------------------------------------------------------------------------------------
VERTICES: 00/03  [=====>>---------------------] 20%   ELAPSED TIME: 6.84 s     
----------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------
        VERTICES      MODE        STATUS  TOTAL  COMPLETED  RUNNI

0: jdbc:hive2://bd-master.newprolab.com:2181,> Closing: 0: jdbc:hive2://bd-master.newprolab.com:2181,bd-node1.newprolab.com:2181,bd-node2.newprolab.com:2181/default;password=dczcdGv5;serviceDiscoveryMode=zooKeeper;user=viktorya.mukukenova;zooKeeperNamespace=hiveserver2


### Output table

In [60]:
grp_hive_tbl_data_path = "/tmp/viktorya.mukukenova/result_lab03_3/result_lab03_1_output_"

create_cmd_line = f"hdfs dfs -mkdir {grp_hive_tbl_data_path}"

!$create_cmd_line

In [61]:
create_hive_tbl = f""" use db_vm;
CREATE EXTERNAL TABLE db_vm.grp_data_output_
(
 URI BIGINT, first_cat BIGINT, second_cat BIGINT, thirst_cat BIGINT, fourth_cat BIGINT
)
row format delimited
fields terminated by '\t'
STORED AS TEXTFILE
LOCATION '{grp_hive_tbl_data_path}';"""

create_script_path = "lab03_create_output__tbl.sql"
with open(create_script_path, 'w') as file_:
    file_.writelines(create_hive_tbl) 
    
cmd_line = f"beeline jdbc:hive2://{host}:{port} -n {login} -p {pwd} -f {create_script_path}"
!$cmd_line

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Connecting to jdbc:hive2://bd-master.newprolab.com:2181,bd-node1.newprolab.com:2181,bd-node2.newprolab.com:2181/default;password=dczcdGv5;serviceDiscoveryMode=zooKeeper;user=viktorya.mukukenova;zooKeeperNamespace=hiveserver2
20/10/24 03:57:43 [main]: INFO jdbc.HiveConnection: Connected to bd-node2.newprolab.com:10000
Connected to: Apache Hive (version 3.1.0.3.1.4.0-315)
Driver: Hive JDBC (version 3.1.0.3.1.4.0-315)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://bd-master.newprolab.com:2181,>  use

In [62]:
insert_hive_tbl = f"""use db_vm; 
INSERT INTO db_vm.grp_data_output_
SELECT A.URI as URI, A.first_cat, A.second_cat, A.thirst_cat, A.fourth_cat
FROM
(
(SELECT URI, first_cat, second_cat, thirst_cat, fourth_cat FROM db_vm.grp_data_unique) 
UNION ALL
(SELECT DISTINCT URI, 0 as first_cat, 0 as second_cat, 0 as thirst_cat, 0 as fourth_cat FROM db_vm.raw_data
where URI NOT IN (SELECT URI FROM db_vm.grp_data_unique))
) A
;
"""

insert_script_path = "lab03_insert_output__tbl.sql"
with open(insert_script_path, 'w') as file_:
    file_.writelines(insert_hive_tbl) 
    
cmd_line = f"beeline jdbc:hive2://{host}:{port} -n {login} -p {pwd} -f {insert_script_path}"
!$cmd_line

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.4.0-315/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Connecting to jdbc:hive2://bd-master.newprolab.com:2181,bd-node1.newprolab.com:2181,bd-node2.newprolab.com:2181/default;password=dczcdGv5;serviceDiscoveryMode=zooKeeper;user=viktorya.mukukenova;zooKeeperNamespace=hiveserver2
20/10/24 03:58:03 [main]: INFO jdbc.HiveConnection: Connected to bd-node2.newprolab.com:10000
Connected to: Apache Hive (version 3.1.0.3.1.4.0-315)
Driver: Hive JDBC (version 3.1.0.3.1.4.0-315)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://bd-master.newprolab.com:2181,> use 

----------------------------------------------------------------------------------------------
        VERTICES      MODE        STATUS  TOTAL  COMPLETED  RUNNING  PENDING  FAILED  KILLED  
----------------------------------------------------------------------------------------------
Map 1            container       RUNNING      2          0        2        0       0       0  
Reducer 3        container        INITED      1          0        0        1       0       0  
Map 6            container        INITED      2          0        0        2       0       0  
Reducer 7        container        INITED      1          0        0        1       0       0  
Reducer 8        container        INITED      2          0        0        2       0       0  
Map 4            container       RUNNING     10          0        4        6       0       0  
Reducer 5        container        INITED      2          0        0        2       0       0  
--------------------------------------------------

----------------------------------------------------------------------------------------------
        VERTICES      MODE        STATUS  TOTAL  COMPLETED  RUNNING  PENDING  FAILED  KILLED  
----------------------------------------------------------------------------------------------
Map 1 .......... container     SUCCEEDED      2          2        0        0       0       0  
Reducer 3        container        INITED      1          0        0        1       0       0  
Map 6            container       RUNNING      2          0        2        0       0       0  
Reducer 7        container        INITED      1          0        0        1       0       0  
Reducer 8        container        INITED      2          0        0        2       0       0  
Map 4            container       RUNNING     10          0       10        0       0       0  
Reducer 5        container        INITED      2          0        0        2       0       0  
--------------------------------------------------

----------------------------------------------------------------------------------------------
        VERTICES      MODE        STATUS  TOTAL  COMPLETED  RUNNING  PENDING  FAILED  KILLED  
----------------------------------------------------------------------------------------------
Map 1 .......... container     SUCCEEDED      2          2        0        0       0       0  
Reducer 3 ...... container     SUCCEEDED      1          1        0        0       0       0  
Map 6 .......... container     SUCCEEDED      2          2        0        0       0       0  
Reducer 7 ...... container     SUCCEEDED      1          1        0        0       0       0  
Reducer 8 ...... container     SUCCEEDED      2          2        0        0       0       0  
Map 4 .......... container     SUCCEEDED     10         10        0        0       0       1  
Reducer 5 ...... container     SUCCEEDED      2          2        0        0       0       0  
--------------------------------------------------

0: jdbc:hive2://bd-master.newprolab.com:2181,> Closing: 0: jdbc:hive2://bd-master.newprolab.com:2181,bd-node1.newprolab.com:2181,bd-node2.newprolab.com:2181/default;password=dczcdGv5;serviceDiscoveryMode=zooKeeper;user=viktorya.mukukenova;zooKeeperNamespace=hiveserver2
